# Huffman Coding

In [1]:
f1="norm_wiki_sample.txt"
with open(f1) as f:
    english=f.read()

## Probability of letters

In [2]:
import copy
def dict_create_letters(text, nr):
    words=[text[i:i+nr] for i in range(len(text)-nr+1)]
    x={}
    for i in words:
        if i[:-1] not in x:
            x[i[:-1]]={}
        x[i[:-1]][i[-1]]=x[i[:-1]].get(i[-1],0)+1
    return x

def normalize(x):
    y=copy.deepcopy(x)
    for key1 in y.keys():
        sumkey1=sum(y[key1].values())
        for key2 in y[key1].keys():
            y[key1][key2]/=sumkey1
    return y

a=normalize(dict_create_letters(english,1))[""]

# Node

In [3]:
class Node:
    probability = None
    letter = None
    code = None
    left = None
    right = None
    
    def __init__(self, a, p) -> None:
        self.probability = p 
        self.letter = a

    def __lt__(self, other):
        return self.probability < other.probability

    def __str__(self) -> str:
        return str(self.letter) + " - " + str(self.probability) + " " + str(self.code)

    def is_leaf(self) -> bool:
        return self.left == self.right == None 

    

# Huffman

In [4]:
import heapq

class Huffman:

    def letter_to_code(self, tree, letter):
        if tree.is_leaf():
            return tree.code
        if letter in tree.left.letter:
            return self.letter_to_code(tree.left, letter)
        else:
            return self.letter_to_code(tree.right, letter)

    def create_queue(self, a):
        priority_queue = []
        for key, value in a.items():
            x = Node([key],value)
            heapq.heappush(priority_queue, x)
        return priority_queue

    def add_codes(self, node, tmp_code):
        if node.is_leaf():
            node.code = tmp_code
        if node.left is not None:
            self.add_codes(node.left, tmp_code + "0")
        if node.right is not None:
            self.add_codes(node.right, tmp_code + "1")

    def create_tree(self, a):
        queue = self.create_queue(a)
        while len(queue) > 1:
            node1 = heapq.heappop(queue)
            node2 = heapq.heappop(queue)
            newNode = Node(node1.letter + node2.letter, node1.probability + node2.probability)
            if node1 > node2:
                newNode.left = node2
                newNode.right = node1
            else:
                newNode.left = node1
                newNode.right = node2
            heapq.heappush(queue, newNode)
        tree = heapq.heappop(queue)
        self.add_codes(tree, "")
        return tree

    def print_codes(self, tree):
        if tree.is_leaf():
            print(tree.letter, " - ", tree.code)
        if tree.left is not None:
            self.print_codes(tree.left)
        if tree.right is not None:
            self.print_codes(tree.right)

    def encode(self, tree, text):
        encoded_text = ""
        for letter in text:
            encoded_text += self.letter_to_code(tree, letter)
        return encoded_text

    def decode(self, tree, text):
        decoded_text = []
        root_tmp = tree
        for i in text:
            if i == "1":
                root_tmp = root_tmp.right     
            else:
                root_tmp = root_tmp.left

            if root_tmp.is_leaf():
                decoded_text.append(root_tmp.letter[0])
                root_tmp = tree
        return decoded_text

# Create tree


In [5]:
huffman = Huffman()
tree = huffman.create_tree(a)


Codes:

In [6]:
huffman.print_codes(tree)

['e']  -  000
['m']  -  00100
['y']  -  001010
['k']  -  0010110
['4']  -  001011100
['x']  -  001011101
['5']  -  001011110
['3']  -  001011111
['s']  -  0011
['w']  -  010000
['b']  -  010001
['c']  -  01001
['r']  -  0101
['o']  -  0110
['n']  -  0111
['i']  -  1000
['d']  -  10010
['2']  -  10011000
['9']  -  10011001
['v']  -  1001101
['g']  -  100111
['t']  -  1010
['p']  -  101100
['f']  -  101101
['l']  -  10111
['a']  -  1100
['h']  -  11010
['8']  -  110110000
['j']  -  110110001
['0']  -  11011001
['q']  -  1101101000
['z']  -  1101101001
['6']  -  1101101010
['7']  -  1101101011
['1']  -  11011011
['u']  -  110111
[' ']  -  111


# Encode

In [7]:
encoded_text = huffman.encode(tree, english)
#print(encoded_text)

Code length vs shortest possible fixed length encoding

In [8]:
print("code length in bits: ", len(encoded_text))
# 2^5 < 37 - characters in our alphabet < 2^6 so shortest fixed_length is 6
print("shortest fixed-length encoding length in bits: ", len(english * 6))
print("ratio: ", len(encoded_text)/ len(english * 6))

code length in bits:  46489714
shortest fixed-length encoding length in bits:  64733646
ratio:  0.718169250037299


# Decode

In [9]:
decoded_text = "".join(huffman.decode(tree, encoded_text))


Checking if correcly decoded:

In [10]:
print(decoded_text == english)

True
